# Build a table: region code -> region polygon

In this section we will build a static reference of region to be used to plot a Colorpleth of France.
Region path will be indexed by region code, the code is retrieved from the INSEE.
We will use these files:
* region2019-csv: columns **Code région** and **Nom en clair (majuscules)**
* regions-version-simplifiee.geojson: The files provided by Mohamed containing regions paths

In [1]:
import json
import pandas as pd

In [2]:
# Download INSEE 2019 region data
!wget https://www.insee.fr/fr/statistiques/fichier/3720946/region2019-csv.zip
!unzip region2019-csv.zip
!ls

'wget' n'est pas reconnu en tant que commande interne
ou externe, un programme ex‚cutable ou un fichier de commandes.
'unzip' n'est pas reconnu en tant que commande interne
ou externe, un programme ex‚cutable ou un fichier de commandes.
'ls' n'est pas reconnu en tant que commande interne
ou externe, un programme ex‚cutable ou un fichier de commandes.


In [10]:
# Load new region codes
region2019_df = pd.read_csv("./region2019.csv")
region2019_df[region2019_df["reg"] ==  27]

,reg,cheflieu,tncc,ncc,nccenr,libelle
7,27,21231,0,BOURGOGNE FRANCHE COMTE,Bourgogne-Franche-Comté,Bourgogne-Franche-Comté


In [57]:
v = region2019_df[region2019_df["reg"] ==  27]["ncc"]

'7    BOURGOGNE FRANCHE COMTE\nName: ncc, dtype: object'

In [7]:
with open("regions-version-simplifiee.geojson") as f:
    geodata = json.load(f)

In [11]:
regions_paths = {}
for f in geodata["features"]:
    code = f["properties"]["code"]
    old_name = f["properties"]["nom"]
    name = region2019_df[region2019_df["reg"] ==  int(code)].iloc[0]["libelle"]
    regions_paths[code] = {
        "name":name,
        "old_name": old_name,
        "feature": f
    }

In [12]:
with open("regions.json", "w") as f:
    json.dump(regions_paths, f)

In [71]:
with open("regions_codes.txt", "w") as f:
    f.write("code\n")
    for code in regions_paths.keys():
        f.write("%s\n" % code)

# Build a table region -> total consumption

We want to have a consumption table. The table would be ideally indexed by INSEE region code so for each region we can retrieve directly its consumption.

In [83]:
regions_codes = pd.read_csv("regions_codes.txt")

In [84]:
df = pd.read_csv("consommation-electrique-par-secteur-dactivite-region.csv", delimiter=";")
df.columns

Index(['Année', 'Nom région', 'Code région', 'Nb sites Résidentiel',
       'Conso totale Résidentiel (MWh)', 'Conso moyenne Résidentiel (MWh)',
       'Nb sites Professionnel', 'Conso totale Professionnel (MWh)',
       'Conso moyenne Professionnel (MWh)', 'Nb sites Agriculture',
       'Conso totale Agriculture (MWh)', 'Nb sites Industrie',
       'Conso totale Industrie (MWh)', 'Nb sites Tertiaire',
       'Conso totale Tertiaire (MWh)', 'Nb sites Secteur non affecté',
       'Conso totale Secteur non affecté (MWh)', 'Nombre d'habitants',
       'Taux de logements collectifs', 'Taux de résidences principales',
       'Superficie des logements < 30 m2',
       'Superficie des logements 30 à 40 m2',
       'Superficie des logements 40 à 60 m2',
       'Superficie des logements 60 à 80 m2',
       'Superficie des logements 80 à 100 m2',
       'Superficie des logements > 100 m2',
       'Résidences principales avant 1919',
       'Résidences principales de 1919 à 1945',
       'Résiden

In [85]:
df.head()

,Année,Nom région,Code région,Nb sites Résidentiel,Conso totale Résidentiel (MWh),Conso moyenne Résidentiel (MWh),Nb sites Professionnel,Conso totale Professionnel (MWh),Conso moyenne Professionnel (MWh),Nb sites Agriculture,...,Résidences principales avant 1919,Résidences principales de 1919 à 1945,Résidences principales de 1946 à 1970,Résidences principales de 1971 à 1990,Résidences principales de 1991 à 2005,Résidences principales de 2006 à 2010,Résidences principales après 2011,Taux de chauffage électrique,Geo Shape,Geo Point 2D
0,2017,Normandie,28,1715164,8.760796e+06,5.107847,231493,2.322050e+06,10.030757,924,...,18.749732,8.027451,22.758724,28.431447,13.204561,6.315746,2.512339,28.874447,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[0....","49.1202642048, 0.11075229648"
1,2017,Centre-Val de Loire,24,1243098,6.644661e+06,5.345243,169765,1.561813e+06,9.199851,4430,...,20.612322,7.641128,19.415312,30.739335,13.688412,5.843834,2.059658,28.256050,"{""type"": ""Polygon"", ""coordinates"": [[[0.614432...","47.4847679099, 1.6843888533"
2,2012,Provence-Alpes-Côte d'Azur,93,2850134,1.458470e+07,5.117198,431940,4.066792e+06,9.415177,1395,...,11.887384,8.437642,26.354425,30.928204,14.597186,5.673194,2.121965,40.334985,NaN,NaN
3,2015,Bourgogne-Franche-Comté,27,1453993,6.691835e+06,4.602385,224364,1.962000e+06,8.744720,779,...,23.215239,8.401511,20.102883,27.404423,13.165538,5.691796,2.018610,17.418529,NaN,NaN
4,2012,Centre-Val de Loire,24,1203311,6.868343e+06,5.707870,168743,1.664268e+06,9.862741,4355,...,20.612322,7.641128,19.415312,30.739335,13.688412,5.843834,2.059658,28.256050,NaN,NaN


In [86]:
df['Agriculture'] = df['Conso totale Agriculture (MWh)']/df['Nb sites Agriculture']
df['Industrie'] = df['Conso totale Industrie (MWh)']/df['Nb sites Industrie']
df['Tertiaire'] = df['Conso totale Tertiaire (MWh)']/df['Nb sites Tertiaire']
df['Autre'] = df['Conso totale Secteur non affecté (MWh)']/df['Nb sites Secteur non affecté']
df['Résidentiel'] = df['Conso moyenne Résidentiel (MWh)'] / (df['Superficie des logements 30 à 40 m2']
                                                                          + df['Superficie des logements 40 à 60 m2']
                                                                          + df['Superficie des logements 60 à 80 m2']
                                                                          + df['Superficie des logements 80 à 100 m2']
                                                                          + df['Superficie des logements < 30 m2']
                                                                          + df['Superficie des logements > 100 m2'])
df['Total'] = (df['Agriculture'] + df['Industrie'] + df['Tertiaire'] + df['Autre'] + df['Résidentiel'])/5



In [87]:
filtered_df = regions_codes.merge(df, right_on="Code région", left_on="code", how="inner")
filtered_df.shape, df.shape, regions_codes.shape
filtered_df = filtered_df[['Année','code',
            'Total',
             'Agriculture',
             'Industrie',
             'Tertiaire',
             'Résidentiel',
             'Autre']]

# Moyenne Consommation

In [88]:
general = 'ConsommationMoyenne'

filtered_df = regions_codes.merge(df, right_on="Code région", left_on="code", how="inner")
filtered_df.shape, df.shape, regions_codes.shape
filtered_df = filtered_df[['Année','code',
            'Total',
             'Agriculture',
             'Industrie',
             'Tertiaire',
             'Résidentiel',
             'Autre']]

availible_years = filtered_df["Année"].unique()
regions_consumption = {general:{}}
for param in ['Total',
             'Agriculture',
             'Industrie',
             'Tertiaire',
             'Résidentiel',
             'Autre']:
    regions_consumption[general][param] = {}
    for code, region_groupdf in filtered_df.groupby("code"):
        regions_consumption[general][param][code] = {}

        for year, year_groupdf in  region_groupdf.groupby("Année"):
            regions_consumption[general][param][code][year] = year_groupdf.iloc[0][param]
            
regions_consumption[general]['histogram'] = 1

# Consommation Total

In [113]:
general = 'ConsommationTotal'
features = ['Conso totale Professionnel (MWh)',
            'Conso totale Agriculture (MWh)',
            'Conso totale Industrie (MWh)',
            'Conso totale Tertiaire (MWh)',
            'Conso totale Résidentiel (MWh)',
            'Conso totale Secteur non affecté (MWh)']

features_renamed = ['Total',
                     'Agriculture',
                     'Industrie',
                     'Tertiaire',
                     'Résidentiel',
                     'Autre']

filtered_df = regions_codes.merge(df, right_on="Code région", left_on="code", how="inner")
filtered_df.shape, df.shape, regions_codes.shape
filtered_df = filtered_df[['Année','code']+features].rename(columns={features[i]:features_renamed[i] for i in range(len(features))})

availible_years = filtered_df["Année"].unique()
regions_consumption[general] = {}
for param in features_renamed:
    regions_consumption[general][param] = {}
    for code, region_groupdf in filtered_df.groupby("code"):
        regions_consumption[general][param][code] = {}

        for year, year_groupdf in  region_groupdf.groupby("Année"):
            regions_consumption[general][param][code][year] = year_groupdf.iloc[0][param].astype(float)
            
regions_consumption[general]['histogram'] = 1

In [90]:
df_prod.columns

Index(['Année', 'Code région', 'Qualite', 'NOM_REGION', 'Production_totale',
       'Production_nucleaire', 'Production_thermique_totale',
       'Production_hydraulique', 'Production_eolien', 'Production_solaire',
       'Production_bioernergies', 'Consommation_totale',
       'Grande_industrie_PME_PMI', 'Energie_industrie_agriculture',
       'Chime_parachimie', 'Construction_automobile', 'Metallurgie_mecanique',
       'Mineraux_materiaux', 'Papier_carton', 'Siderurgie',
       'Autres_industries', 'Excedent_Deficit'],
      dtype='object')

In [115]:
df_prod = pd.read_csv('donnees_economix.csv', sep=',')
df_prod = df_prod.rename(columns={'annee':'Année','code':'Code région'})

general = 'ProductionTotale'
features = ['Production_totale',
            'Production_nucleaire',
            'Production_thermique_totale',
            'Production_hydraulique',
            'Production_eolien',
            'Production_solaire',
            'Production_bioernergies',]

features_renamed = ['Total',
                    'Nucleaire',
                    'Thermique',
                    'Hydraulique',
                    'Eolien',
                    'Solaire',
                    'Bioernergies']

filtered_df = regions_codes.merge(df_prod, right_on="Code région", left_on="code", how="inner")
filtered_df.shape, df_prod.shape, regions_codes.shape
filtered_df = filtered_df[['Année','code']+features].rename(columns={features[i]:features_renamed[i] for i in range(len(features))})

availible_years = filtered_df["Année"].unique()
regions_consumption[general] = {}
for param in features_renamed:
    regions_consumption[general][param] = {}
    for code, region_groupdf in filtered_df.groupby("code"):
        regions_consumption[general][param][code] = {}

        for year, year_groupdf in  region_groupdf.groupby("Année"):
            regions_consumption[general][param][code][year] = year_groupdf.iloc[0][param].astype(float)
            
regions_consumption[general]['histogram'] = 1

# Production CO2

In [116]:
filtered_df['Total'] = filtered_df['Nucleaire']*6 + filtered_df['Thermique']*900 + filtered_df['Hydraulique']*4 + \
                        filtered_df['Eolien']*10 + filtered_df['Solaire']*100


In [117]:
general = 'ProductionCO2'
regions_consumption[general] = {}

for param in ['Total']:
    regions_consumption[general][param] = {}
    for code, region_groupdf in filtered_df.groupby("code"):
        regions_consumption[general][param][code] = {}

        for year, year_groupdf in  region_groupdf.groupby("Année"):
            regions_consumption[general][param][code][year] = year_groupdf.iloc[0][param].astype(float)
                          
regions_consumption[general]['histogram'] = 0

# Excedant

In [121]:
filtered_df['Total'] = df_prod['Excedent_Deficit']

general = 'Excedent'
regions_consumption[general] = {}

for param in ['Total']:
    regions_consumption[general][param] = {}
    for code, region_groupdf in filtered_df.groupby("code"):
        regions_consumption[general][param][code] = {}

        for year, year_groupdf in  region_groupdf.groupby("Année"):
            regions_consumption[general][param][code][year] = year_groupdf.iloc[0][param].astype(float)
                          
regions_consumption[general]['histogram'] = 0

In [123]:
for elmt in regions_consumption.keys():
    with open("all_consumption.json", "w") as f:
        json.dump(regions_consumption, f)

In [119]:
regions_consumption

{'ConsommationMoyenne': {'Total': {11: {2011: 202.22450934787628,
    2012: 188.13839929366003,
    2013: 194.72582068661055,
    2014: 170.5037198626384,
    2015: 180.50532972524917,
    2016: 223.93573141902334,
    2017: 167.04781758381677},
   24: {2011: 295.2770093079883,
    2012: 246.62555751448204,
    2013: 282.17838131618197,
    2014: 226.20914268814704,
    2015: 277.9350984710667,
    2016: 248.37671220487147,
    2017: 229.3311779904139},
   27: {2011: 251.0817201207558,
    2012: 236.4009481121794,
    2013: 241.41971305114595,
    2014: 225.91058481093842,
    2015: 237.49797441630753,
    2016: 261.8803225745466,
    2017: 230.8142950356781},
   28: {2011: 310.72554084214045,
    2012: 296.4708399175207,
    2013: 292.03580045051353,
    2014: 282.67535006092413,
    2015: 288.37675807201174,
    2016: 855.3146145475584,
    2017: 284.76951856138686},
   32: {2011: 384.06754644601034,
    2012: 367.346547084478,
    2013: 363.6901700491325,
    2014: 348.6308123345737